In [13]:
import pandas as pd
import numpy as np

<h1>Part one</h1>

In [30]:
with open('postal_code.txt') as f:
     lines = [ line.strip( ) for line in list(f) ]
x =  len(lines)//4
lines[0:3]

['Postal Code', 'Borough', 'Neighbourhood']

**set up column names**

In [31]:
df = pd.DataFrame(columns=lines[0:3]) 
df

,Postal Code,Borough,Neighbourhood


**drop column names from lines**

In [32]:
lines = lines[3:] 
lines[0:3]

['M1A', 'Not assigned', 'Not assigned']

In [33]:
f.close()

**split lines to columns**

In [34]:
p_code = []
borough = []
neigh = []
for i in range(len(lines)): 
    if i%3 == 0:
        p_code.append(lines[i])
    if i%3 == 1:
        borough.append(lines[i])
    if i%3 == 2:
        neigh.append(lines[i])

**drop boroughs that are not equal to not assigned**

In [35]:
df['Postal Code'] = p_code
df['Borough'] = borough
df['Neighbourhood'] = neigh
df = df[df['Borough']!='Not assigned'] 

In [36]:
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [37]:
df.shape

(103, 3)

<h1>Part two</h1>

In [41]:
#!pip install geocoder

In [53]:
import geocoder
geo = pd.read_csv('Geospatial_Coordinates.csv')
geo.tail()

,Postal Code,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [52]:
df.sort_values(by = 'Postal Code', inplace = True)
df.tail()

<ipython-input-52-bafd564d6395>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by = 'Postal Code', inplace = True)


,Postal Code,Borough,Neighbourhood,Latitude
98,M9N,York,Weston,43.706876
107,M9P,Etobicoke,Westmount,NaN
116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",NaN
143,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",NaN
152,M9W,Etobicoke,"Northwest, West Humber - Clairville",NaN


In [55]:
df['Latitude'] = geo['Latitude']
df['Longitude'] = geo['Longitude']
df.head()

<ipython-input-55-997202e044df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Latitude'] = geo['Latitude']
<ipython-input-55-997202e044df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Longitude'] = geo['Longitude']


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern, Rouge",43.692657,-79.264848
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.778517,-79.346556
27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.725900,-79.340923
36,M1G,Scarborough,Woburn,43.695344,-79.318389
45,M1H,Scarborough,Cedarbrae,43.712751,-79.390197


In [67]:
df = df.dropna(subset=['Latitude'])
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern, Rouge",43.692657,-79.264848
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.778517,-79.346556
27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.725900,-79.340923
36,M1G,Scarborough,Woburn,43.695344,-79.318389
45,M1H,Scarborough,Cedarbrae,43.712751,-79.390197
...,...,...,...,...,...
17,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.803762,-79.363452
26,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.745906,-79.352188
80,M9L,North York,Humber Summit,43.691116,-79.476013
89,M9M,North York,"Humberlea, Emery",43.602414,-79.543484


<h1>Part three</h1>

In [57]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [58]:
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [69]:
# create map of New York using latitude and longitude values
map_ = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_)  
    
map_